In [1]:
import os
import pyspark

from pymongo import MongoClient

In [2]:
conf = pyspark.SparkConf()
conf.set('spark.ui.proxyBase', '/user/' + os.environ['JUPYTERHUB_USER'] + '/proxy/4040')

sc = pyspark.SparkContext(conf=conf)
spark = pyspark.sql.SparkSession(sc)
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/08 23:31:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
username = "vt2182"
password = "vt2182"
host = "mongo-csgy-6513-spring.db"
auth_database = "vt2182"

In [4]:
mongo_conn_str = f"mongodb://{username}:{password}@{host}/{auth_database}"
client = MongoClient(mongo_conn_str)

In [5]:
db = client[auth_database]
db

Database(MongoClient(host=['mongo-csgy-6513-spring.db:27017'], document_class=dict, tz_aware=False, connect=True), 'vt2182')

In [6]:
db.list_collection_names()

['yellow_tripdata_raw']

In [7]:
import os
import glob
import pandas as pd

In [9]:
# Define the path where your parquet files are stored
path = "../data/"

# Use glob to match the pattern ‘*.parquet’
files = glob.glob(os.path.join(path, "*.parquet"))

# Iterate over the list of files
for file in files:
    # Load the parquet file
    df = pd.read_parquet(file)

    # Check if the data frame has more than 10,000 rows
    if len(df) > 10000:
        # Randomly select 10,000 rows
        df_sample = df.sample(n=10000)
    else:
        print(f"The dataset in {file} has less than 10,000 rows.")
        continue

    # Overwrite the parquet file with the sampled data
    df_sample.to_parquet(file)

In [10]:
# Load the data
df = spark.read.parquet('../data/yellow_tripdata_*.parquet')

In [11]:
df.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- __index_level_0__: long (nullable = true)



In [12]:
df.head()

Row(VendorID=2, tpep_pickup_datetime=datetime.datetime(2022, 12, 3, 19, 14, 47), tpep_dropoff_datetime=datetime.datetime(2022, 12, 3, 19, 31, 51), passenger_count=1.0, trip_distance=3.37, RatecodeID=1.0, store_and_fwd_flag='N', PULocationID=143, DOLocationID=152, payment_type=1, fare_amount=13.5, extra=0.5, mta_tax=0.5, tip_amount=3.46, tolls_amount=0.0, improvement_surcharge=0.3, total_amount=20.76, congestion_surcharge=2.5, airport_fee=0.0, __index_level_0__=375653)

In [13]:
df.show(5)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+-----------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|__index_level_0__|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+-----------------+
|       2| 2022-12-03 19:14:47|  2022-12-03 19:31:51|            1.0|         3.37|       1.0|                 N|         143|       

In [14]:
# Count the number of rows
print("The data contains {} rows.".format(df.count()))

The data contains 120000 rows.


In [15]:
# Get the collection
collection = db['yellow_tripdata_raw']

In [16]:
# Convert the PySpark DataFrame to a Pandas DataFrame
pandas_df = df.toPandas()

/opt/conda/envs/bigdata/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)
/opt/conda/envs/bigdata/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


In [17]:
# Convert the Pandas DataFrame to a dictionary
data_dict = pandas_df.to_dict("records")

In [19]:
# Insert the dictionary into MongoDB
collection.insert_many(data_dict)